<a href="https://colab.research.google.com/github/grifire/tp-rag-student-version/blob/main/rendu.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import os

# Vérifie si le code est exécuté sur Google Colab
if 'COLAB_GPU' in os.environ:
    # Commandes à exécuter uniquement sur Google Colab
    !git clone https://github.com/grifire/tp-rag-student-version.git
    %cd tp-rag-student-version
    !pip install -r requirement.txt
else:
    # Commandes à exécuter si ce n'est pas sur Google Colab
    print("Pas sur Google Colab, ces commandes ne seront pas exécutées.")

##  Etape 1. - Indexation des documents

In [ ]:
pip install -qU langchain-huggingface

In [ ]:
import torch.nn.functional as F

from torch import Tensor
from transformers import AutoTokenizer, AutoModel
from langchain_huggingface import HuggingFaceEmbeddings

embeddings = HuggingFaceEmbeddings(model_name="intfloat/multilingual-e5-base")

In [ ]:
rm -rf chroma_langchain_db


In [ ]:
pip install -qU langchain-chroma

In [ ]:
from langchain_chroma import Chroma


vector_store = Chroma(
    collection_name="example_collection",
    embedding_function=embeddings,
    persist_directory="./chroma_langchain_db",  # Where to save data locally, remove if not necessary
)

In [ ]:
pip install -qU langchain-community pypdf pillow

### Exercice 1 Indexation

In [ ]:
from langchain_community.document_loaders import PyPDFDirectoryLoader
loader = PyPDFDirectoryLoader("data/")
#loader2 (txt)
#loader3 (latex)
docs = loader.load() # + loarder2.load() + loader3.load()

print(f"number of documents: {len(docs)}")
# assert len(docs) == 1
print(f"Total characters: {len(docs[0].page_content)}")

In [ ]:
from langchain_text_splitters import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1000,  # chunk size (characters)
    chunk_overlap=200,  # chunk overlap (characters)
    add_start_index=True,  # track index in original document
)
all_splits = text_splitter.split_documents(docs)

print(f"Split blog post into {len(all_splits)} sub-documents.")

In [ ]:
document_ids = vector_store.add_documents(documents=all_splits)

print(document_ids[:3])

In [ ]:
docs[0]

### Exercice 2 Interrogation

In [ ]:
def retrieve_context(query: str):
    """Retrieve information to help answer a query."""
    retrieved_docs = vector_store.similarity_search(query, k=2)
    serialized = "\n\n".join(
        (f"Source: {doc.metadata}\nContent: {doc.page_content}")
        for doc in retrieved_docs
    )
    return serialized, retrieved_docs

In [ ]:
res_query = retrieve_context('Attention is all you need')
print(f"Résultat : {res_query}")

In [ ]:
def retrieve_context_score(query: str):
  retrieved_docs = []
  scores = []
  for res, score in vector_store.similarity_search_with_relevance_scores("Attention is all you need", k=2) :
    retrieved_docs.append(res)
    scores.append(score)
  serialized = "\n\n".join(
        (f"Source: {doc.metadata}\nContent: {doc.page_content}")
        for doc in retrieved_docs
  )
  return serialized, retrieved_docs, scores
#print(score)

# for doc, score in res:
#     print(doc.metadata)
#     print(score)

In [ ]:
res_query_score = retrieve_context_score('Attention is all you need')
for i in range(len(res_query_score[1])) :
  print(f"Score de {res_query_score[2][i]} :\n{res_query_score[1][i]}\n\n")


## Etape 2

### Exercice 3

In [ ]:
from langchain_core.prompts import ChatPromptTemplate
prompt = ChatPromptTemplate.from_messages([
(
  "system",
  "You are an experienced journalist. "
  "Answer the user's question using ONLY the provided context. "
  "Cite the relevant excerpts from the context in your answer."
),
(
  "human",
  "Context:\n{context}\n\nQuestion:\n{question}"
)
])

### Exercice 4

In [ ]:
!pip install colab-xterm
%load_ext colabxterm
%xterm

In [ ]:
pip install -qU langchain-ollama

In [ ]:
from langchain_ollama import ChatOllama

llm = ChatOllama(
    model="qwen3:8b",
    temperature=0,
)

In [ ]:
messages = [
    (
        "system",
        "You are a helpful assistant that translates English to French. Translate the user sentence.",
    ),
    ("human", "I love programming."),
]
ai_msg = llm.invoke(messages)
ai_msg

In [ ]:
def PromptTemplate(retrieve, question, langue = "french"):
  prompt = [
      (
          "system",
          f"You are a helpful assistant. Answer in the following language : {langue}"
      ),
      (
          "human",
          f"Context:\n{retrieve(question)[0]}\n\nQuestion:\n{question}"
      )
  ]
  return prompt

In [ ]:
PromptTemplate(retrieve_context_score,"What is attention?")

In [ ]:
ai_message = llm.invoke(PromptTemplate(retrieve_context_score,"What is the attention"))
ai_message

In [ ]:
#Modifié par gemini.Semble marcher mais a vérifier
from langgraph.checkpoint.memory import InMemorySaver

llm_agent = ChatOllama(
    model="qwen3:8b",
    temperature=0,
    memory=InMemorySaver(),
)

llm_agent.invoke(
    [{"role": "user", "content": "Hi! My name is Bob."}] ,
    config= {"configurable": {"thread_id": "1"}},
)